# **<a id="Content">HnM RecSys Notebook 9417</a>**

## **<a id="Content">Table of Contents</a>**
* [**<span>1. Imports</span>**](#Imports)  
* [**<span>2. Helper Functions/Decorators</span>**](#Helper-Functions)
* [**<span>5. LightGBM Model</span>**](#LightGBM-Model) 

## Imports

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import os
import re
import warnings
import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
# import cudf # switch on P100 GPU for this to work in Kaggle
# import cupy as cp

# Importing data
articles = pd.read_csv('articles.csv')
print(articles.head())
print("--")
customers = pd.read_csv('customers.csv')
print(customers.head())
print("--")
transactions = pd.read_csv("transactions_train.csv")
print(transactions.head())
print("--")

   article_id  product_code          prod_name  product_type_no   
0   108775015        108775          Strap top              253  \
1   108775044        108775          Strap top              253   
2   108775051        108775      Strap top (1)              253   
3   110065001        110065  OP T-shirt (Idro)              306   
4   110065002        110065  OP T-shirt (Idro)              306   

  product_type_name  product_group_name  graphical_appearance_no   
0          Vest top  Garment Upper body                  1010016  \
1          Vest top  Garment Upper body                  1010016   
2          Vest top  Garment Upper body                  1010017   
3               Bra           Underwear                  1010016   
4               Bra           Underwear                  1010016   

  graphical_appearance_name  colour_group_code colour_group_name  ...   
0                     Solid                  9             Black  ...  \
1                     Solid               

In [18]:
# LightGBM imports

from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.feature_selection import RFE
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import make_scorer
import lightgbm as lgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
import pandas as pd

In [4]:
import pickle

# open user_item_matrix_200
with open('user_item_matrix_200.pkl', 'rb') as f:
    user_item_matrix = pickle.load(f)

# open customer and articels incides map
with open('lightgbm/customer_id_indices_map.pkl', 'rb') as f:
    customer_id_indices_map = pickle.load(f)

with open('lightgbm/article_id_indices_map.pkl', 'rb') as f:
    article_id_indices_map = pickle.load(f)

# load df from pickle file for time-based split
with open('lightgbm/df.pkl', 'rb') as f:
    df = pickle.load(f)

# load final_df from pickle file for clean processing
with open('lightgbm/final_df_with_binary_targets.pkl', 'rb') as f:
    final_df = pickle.load(f)

In [5]:
final_df.shape

(7803051, 56)

In [6]:
df.shape

(126622, 36)

In [8]:
# get top 200 customers by number of transactions
top_customers = transactions['customer_id'].value_counts().head(200).index.tolist()

# print num of transactions for the 200th customer
print(transactions['customer_id'].value_counts().sort_values(ascending=False).iloc[199])

# only get articles that were purchased by top 200 customers at least once in articles df
articles_top_200 = articles[articles['article_id'].isin(transactions[transactions['customer_id'].isin(top_customers)]['article_id'].unique())]

# only get 200 customers in customers df
customers_top_200 = customers[customers['customer_id'].isin(top_customers)]

articles = articles_top_200.copy()
customers = customers_top_200.copy()
transactions = transactions[transactions['customer_id'].isin(top_customers)].copy()
transactions = transactions.drop_duplicates().copy()

622


In [10]:
customers_final_df = customers.copy()
transactions_final_df = transactions.copy()

# Merge transactions with customers
df = pd.merge(transactions_final_df, customers_final_df, on='customer_id', how='left')

# Merge resulting dataframe with articles_final_df usually
df = pd.merge(df, articles, on='article_id', how='left')

In [12]:
df.shape

(126709, 35)

In [19]:
import pandas as pd

transactions_df = transactions.copy()

transactions_df = transactions_df.drop_duplicates(subset=['customer_id','article_id'])
transactions_df = transactions_df.reset_index(drop=True)

customer_list = transactions_df.customer_id.value_counts()[:200].index.tolist()

customers_1 = customers.loc[customers['customer_id'].isin(customer_list)]
transactions_train_t = transactions_df.loc[transactions_df['customer_id'].isin(customer_list)]

articles_t = articles.loc[articles['article_id'].isin(list(transactions_train_t['article_id']))]

merge_df = pd.DataFrame(columns=['customer_id', 'Buy', 'article_id'])

for customer in customers_1['customer_id']:
    new_df = pd.DataFrame()
    new_df['Buy'] = pd.Series(dtype='int64')
    new_df = new_df.append([customers_1.loc[customers_1['customer_id'] == customer]] * len(articles_t)).reset_index(drop=True)
    new_df['article_id'] = list(articles_t['article_id'])
    tran_temp = transactions_train_t.loc[transactions_train_t['customer_id'] == customer]
    for article in list(tran_temp['article_id']):
        new_df.loc[(new_df['customer_id'] == customer) & (new_df['article_id'] == article), ['Buy']] = 1
    merge_df = merge_df.append(new_df).reset_index(drop=True)

merge_df['Buy'] = merge_df['Buy'].astype('int64')

0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37e011580a479e80aa94


AttributeError: 'DataFrame' object has no attribute 'append'